# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


## Initialize the Session

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

# Initialize Glue Context
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: 79b238aa-1903-4066-9b56-a979eaf981cd
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 79b238aa-1903-4066-9b56-a979eaf981cd to get into ready status...
Session 79b238aa-1903-4066-9b56-a979eaf981cd has been created.



In [9]:
from pyspark.sql.functions import col, from_unixtime, to_timestamp
import pyspark.sql.functions as f

In [6]:
spark.conf.set("spark.sql.caseSensitive", "true")

## Read and Filter the Reviews Dataset

In [4]:
# Replace with your S3 paths
reviews_input = "s3://recommendation-project-rapid/raw/All_Beauty.jsonl"

# Load Reviews
df_reviews = spark.read.json(reviews_input)

# Select only the columns needed for the recommendation engine
# We use 'parent_asin' as the join key
reviews_cleaned = (
    df_reviews.select(
    col("user_id"),
    col("parent_asin"),
    col("rating").cast("double"),
    col("timestamp") #.cast("long").alias("event_time")
    )
    .withColumn(
        "event_time_seconds", (col("timestamp") / 1000).cast("double")
    ).withColumn(
        "calendar_date", from_unixtime(col("event_time_seconds"), "yyyy-MM-dd")
    )
    .drop("timestamp")
)

In [13]:
reviews_cleaned.select(f.min("calendar_date"), f.max("calendar_date")).show()

+------------------+------------------+
|min(calendar_date)|max(calendar_date)|
+------------------+------------------+
|        2000-11-01|        2023-09-09|
+------------------+------------------+


In [15]:
df_reviews.count()

701528


## Read and Filter the Metadata Dataset

In [7]:
metadata_input = "s3://recommendation-project-rapid/raw/meta_All_Beauty.jsonl"

# Load Metadata
df_metadata = spark.read.json(metadata_input)

# Select only the columns needed
metadata_cleaned = df_metadata.select(
    col("parent_asin").alias("meta_parent_asin"), # Avoid name collision during join
    col("title").alias("movie_title"),
    col("main_category")
)

## Join the Datasets

In [8]:
# Join on parent_asin
final_df = reviews_cleaned.join(
    metadata_cleaned, 
    reviews_cleaned.parent_asin == metadata_cleaned.meta_parent_asin, 
    "inner"
).drop("meta_parent_asin") # Drop the duplicate key

# Preview the joined result
final_df.show(5)

+--------------------+-----------+------+------------------+-------------+--------------------+-------------+
|             user_id|parent_asin|rating|event_time_seconds|calendar_date|         movie_title|main_category|
+--------------------+-----------+------+------------------+-------------+--------------------+-------------+
|AGKHLEW2SOWHNMFQI...| B00YQ6X8EO|   5.0|  1.588687728923E9|   2020-05-05|Herbivore - Natur...|   All Beauty|
|AGKHLEW2SOWHNMFQI...| B081TJ8YS3|   4.0|   1.58861585507E9|   2020-05-04|All Natural Vegan...|   All Beauty|
|AE74DYR3QUGVPZJ3P...| B097R46CSY|   5.0|  1.589665266052E9|   2020-05-16|New Road Beauty -...|   All Beauty|
|AFQLNQNQYFWQZPJQZ...| B09JS339BZ|   1.0|   1.64339363022E9|   2022-01-28|muaowig Ombre Bod...|   All Beauty|
|AFQLNQNQYFWQZPJQZ...| B08BZ63GMJ|   5.0|  1.609322563534E9|   2020-12-30|Yinhua Electric N...|   All Beauty|
+--------------------+-----------+------+------------------+-------------+--------------------+-------------+
only showi

In [10]:
final_df.select(f.min("calendar_date"), f.max("calendar_date")).show()

+------------------+------------------+
|min(calendar_date)|max(calendar_date)|
+------------------+------------------+
|        2000-11-01|        2023-09-09|
+------------------+------------------+


## Save as Optimized Parquet

In [17]:
output_path = "s3://recommendation-project-rapid/processed/all_beauty_dataset/"

# Write to S3 in Parquet format with Snappy compression
final_df.write \
    .mode("overwrite") \
    .partitionBy("rating") \
    .parquet(output_path)

print(f"Job Complete! Joined data saved to: {output_path}")

Job Complete! Joined data saved to: s3://recommendation-project-rapid/processed/all_beauty_dataset/
